# PneumoDetect Week 2 — Performance Review & Clinical Report v1

**Notebook:** `05_performance_review.ipynb`  
**Focus:** Quantitative evaluation and clinical interpretability of Grad-CAM explainability results.

---

## Objective
Evaluate the diagnostic performance and explainability quality of PneumoDetect models (baseline, balanced, and fine-tuned ResNet-50 variants). The notebook generates evaluation figures, Grad-CAM montages, and a structured performance report for clinical interpretation.

---

## Workflow Overview

| Step | Description | Output |
|------|--------------|---------|
| 1 | Load trained models (`resnet50_baseline.pt`, `resnet50_best.pt`, `resnet50_finetuned.pt`) | Model checkpoints verified |
| 2 | Generate ROC and Precision–Recall curves | `roc_pr_curves.png` |
| 3 | Produce Grad-CAM overlays and montages | `gradcam_montage.png` |
| 4 | Compute AUC, Sensitivity, Specificity | `performance_metrics.npy` |
| 5 | Export clinical summary report | `docs/performance_report_v1.md` |

---


In [1]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
# ------------------------------------------------------------
# 1. Setup
# ------------------------------------------------------------
import torch, numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from pathlib import Path
import seaborn as sns
from src.gradcam import generate_cam, GradCAM

model_path = Path("saved_models/resnet50_best.pt")
data_dir = Path("data/rsna_subset/val_images")
report_dir = Path("reports/week2_performance_review")
report_dir.mkdir(parents=True, exist_ok=True)
